<a href="https://colab.research.google.com/github/meti-94/BERT-QA/blob/main/Freebase2Mtranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/castorini/BuboQA.git

Cloning into 'BuboQA'...
remote: Enumerating objects: 1383, done.
remote: Total 1383 (delta 0), reused 0 (delta 0), pack-reused 1383
Receiving objects: 100% (1383/1383), 361.07 KiB | 20.06 MiB/s, done.
Resolving deltas: 100% (908/908), done.


In [ ]:
!pip install fuzzywuzzy
%cd ./BuboQA
!bash setup.sh 


/content/BuboQA
Running fetch_dataset script

--2021-05-24 18:26:51--  https://git.uwaterloo.ca/jimmylin/BuboQA-data/raw/master/SimpleQuestions_v2.tgz
Resolving git.uwaterloo.ca (git.uwaterloo.ca)... 129.97.83.4
Connecting to git.uwaterloo.ca (git.uwaterloo.ca)|129.97.83.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 423435590 (404M) [application/x-gzip]
Saving to: ‘SimpleQuestions_v2.tgz’

SimpleQuestions_v2. 100%[===================>] 403.82M   101MB/s    in 4.2s    

2021-05-24 18:27:00 (97.2 MB/s) - ‘SimpleQuestions_v2.tgz’ saved [423435590/423435590]



Unzipping SimpleQuestions dataset...

tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
./._SimpleQuestions_v2
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended head

In [2]:
import pickle
with open('/content/drive/MyDrive/qa/names_2M.pkl', 'rb') as fin:
    mid2name = pickle.load(fin)
with open('/content/drive/MyDrive/qa/entity_2M.pkl', 'rb') as fin:
    ent2mid = pickle.load(fin)
with open('/content/drive/MyDrive/qa/reachability_2M.pkl', 'rb') as fin:
    reachability = pickle.load(fin)
with open('/content/drive/MyDrive/qa/degrees_2M.pkl', 'rb') as fin:
    degrees = pickle.load(fin)
    


In [ ]:
!cp /content/BuboQA/indexes/names_2M.pkl /content/drive/MyDrive/qa/
!cp /content/BuboQA/indexes/entity_2M.pkl /content/drive/MyDrive/qa/
!cp /content/BuboQA/indexes/reachability_2M.pkl /content/drive/MyDrive/qa/
!cp /content/BuboQA/indexes/degrees_2M.pkl /content/drive/MyDrive/qa/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(mid2name['fb:m.0n1vy1h'], mid2name['fb:m.05zppz'])

['brandon kramer'] ['dude', 'guy', 'male', 'man', 'men', 'boy']


In [ ]:
print(len(degrees), list(degrees.keys())[:24], degrees['fb:m.0n1vy1h'], degrees['fb:soccer.football_player.position_s'])

1968568 ['fb:m.0n1vy1h', 'fb:people.person.gender', 'fb:m.05zppz', 'fb:m.038b5bg', 'fb:music.release.track', 'fb:m.010btzs', 'fb:m.0cz9079', 'fb:soccer.football_player.position_s', 'fb:m.02nzb8', 'fb:m.03d1n81', 'fb:organization.organization_founder.organizations_founded', 'fb:m.04hk45', 'fb:m.03byqr1', 'fb:cvg.computer_videogame.cvg_genre', 'fb:m.0_678', 'fb:m.0n5xzdf', 'fb:people.person.place_of_birth', 'fb:m.0c_m3', 'fb:m.02vwbkk', 'fb:people.person.nationality', 'fb:m.01mk6', 'fb:m.02hl5l7', 'fb:cvg.game_version.platform', 'fb:m.050xd'] [0, 5] [54191, 0]


In [ ]:
print(len(mid2name), list(mid2name.keys())[:20], mid2name['fb:m.01jp8ww'], mid2name['fb:m.01qzt1'])

1951909 ['fb:m.0f8vjgd', 'fb:m.0f8vp8r', 'fb:m.0f8vt9', 'fb:m.0f8w8td', 'fb:m.0f8xqp', 'fb:m.0f966w9', 'fb:m.0f97_j', 'fb:m.0f98_c1', 'fb:m.0f9k3d7', 'fb:m.0f9mxbc', 'fb:m.0f9q09k', 'fb:m.0f9qpn9', 'fb:m.0f9rv4n', 'fb:m.0f9x06p', 'fb:m.0f_3k2', 'fb:m.0f_44r3', 'fb:m.0f_61lf', 'fb:m.0f_6gvn', 'fb:m.0f__g6', 'fb:m.0f_dj4_'] ['harder ... ..faster', 'harder ... faster'] ['classic rock']


In [ ]:
print(len(reachability), list(reachability.keys())[:10], reachability['fb:m.01jp8ww'])

1959805 ['fb:m.0n1vy1h', 'fb:m.038b5bg', 'fb:m.0cz9079', 'fb:m.03d1n81', 'fb:m.03byqr1', 'fb:m.0n5xzdf', 'fb:m.02vwbkk', 'fb:m.02hl5l7', 'fb:m.0b70yxd', 'fb:m.0nd_0ll'] {'fb:music.album.genre', 'fb:music.album.artist', 'fb:music.album.release_type'}


In [ ]:
print(len(ent2mid), list(ent2mid.keys())[:10])

4798616 ['skyshaper', 'collection', 'gypsy 83', 'gypsy', '83', 'cast )', 'london', '1971 london', 'show boat (', '(']


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

subject_vectorizer = TfidfVectorizer()
all_entities = [' '.join(item) for item in mid2name.values()]
all_entity_vectors = subject_vectorizer.fit_transform(all_entities)
# mid2vector = {i:j for i,j in zip(mid2name.keys(), all_entity_vectors)}

predicate_vectorizer = TfidfVectorizer()
all_predicates = [item.split('.')[-1].replace('_', ' ') for item in list(degrees.keys())[1::3]]
all_predicate_vectors = predicate_vectorizer.fit_transform(all_predicates)
# pred2vector = {i:j for i,j in zip(all_predicates, all_predicate_vectors)}

In [6]:
# del all_entities; del all_predicates
# del mid2vector; del pred2vector

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
def query(subject_list, predicate_list, 
          subject_vectorizer, subject_vectors, 
          predicate_vectorizer, ent_cand_number):
  subject_similarities = cosine_similarity(subject_vectors, subject_vectorizer.transform(subject_list))
  results = []
  for index in range(subject_similarities.shape[-1]):
    cands = []
    mid2similarity = {i:j for i,j in zip(mid2name.keys(), subject_similarities[:, index])}
    
    sorted_mid2similarity = {k: [v, reachability[k]] for k, v in sorted(mid2similarity.items(), key=lambda item: item[1], reverse=True)[:ent_cand_number]}
    print(sorted_mid2similarity)
    print('='*100)
    candidates = []
    for mid, value in sorted_mid2similarity.items():
      for prd in value[1]:
        temp = cosine_similarity(predicate_vectorizer.transform([prd.split('.')[-1].replace('_', ' ')]),
                                      predicate_vectorizer.transform(predicate_list[index:index+1])).flatten()[0]
        candidates.append([mid, value[0], prd, temp])     
    sorted_candidates = sorted(candidates, key=lambda item:item[1]+item[3], reverse=True)
    graph = list(degrees.keys())
    for item in sorted_candidates:
      indices = [i for i, x in enumerate(graph) if x == item[0]]
      for index in indices:
        if graph[index+1]==item[2]:
          # item.append(graph[index+2])
          cands.append(item+[graph[index+2]])
    results.append(cands)
  return results

    



In [8]:

sss = query(['brandon kramer', 'praying for time'], ['gender', 'composer'], subject_vectorizer, all_entity_vectors, predicate_vectorizer, 10)

{'fb:m.0n1vy1h': [1.0, {'fb:people.person.profession', 'fb:people.person.gender', 'fb:common.topic.notable_types'}], 'fb:m.0873tb': [0.7108760682150832, {'fb:people.person.gender', 'fb:common.topic.notable_types', 'fb:people.person.nationality', 'fb:music.producer.releases_produced', 'fb:music.artist.album', 'fb:people.person.profession', 'fb:music.musical_group.member', 'fb:music.performance_role.regular_performances', 'fb:music.artist.track', 'fb:people.person.place_of_birth', 'fb:music.artist.label', 'fb:organization.organization_founder.organizations_founded'}], 'fb:m.0p_th': [0.7081996632348486, {'fb:film.film.music', 'fb:film.film.country', 'fb:common.topic.notable_types', 'fb:film.film.film_set_decoration_by', 'fb:film.film.story_by', 'fb:film.film.language', 'fb:award.award_nominee.award_nominations', 'fb:film.film.genre', 'fb:award.award_nominated_work.award_nominations', 'fb:film.film.soundtrack', 'fb:film.film.rating', 'fb:film.film.directed_by', 'fb:film.film.written_by', '

In [ ]:
sss

[[['fb:m.0n1vy1h', 1.0, 'fb:people.person.gender', 1.0, 'fb:m.05zppz']],
 [['fb:m.08zwyb',
   1.0000000000000002,
   'fb:music.composition.composer',
   1.0,
   'fb:m.01vvyfh']]]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
a = cosine_similarity(all_entity_vectors, subject_vectorizer.transform(['choir','orchestra , tommy']*2))
# for i in range(0, 1000):
# print(cosine_similarity(mid2vector['fb:m.0f__g6'], entity_vectorizer.transform(['orange lake , florida orange lake'])).flatten())

In [ ]:
a[:10]

array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.4809354 , 0.55172393, 0.4809354 , 0.55172393],
       [0.        , 0.        , 0.        , 0.        ]])

In [ ]:
subject_vectorizer.inverse_transform(all_entity_vectors[:30])

[array(['skyshaper'], dtype='<U70'),
 array(['collection'], dtype='<U70'),
 array(['83', 'gypsy'], dtype='<U70'),
 array(['cast', 'revival', 'london', '1971', 'boat', 'show'], dtype='<U70'),
 array(['surfers', 'movie'], dtype='<U70'),
 array(['winterreise', 'die'], dtype='<U70'),
 array(['quarterly', 'rights', 'human'], dtype='<U70'),
 array(['me', 'remember'], dtype='<U70'),
 array(['choir', 'chamber', 'orchestra', 'symphony', 'tommy', 'london'],
       dtype='<U70'),
 array(['that', 'smack'], dtype='<U70'),
 array(['having', 're', 'we', 'weather', 'lovely'], dtype='<U70'),
 array(['city', 'kansas'], dtype='<U70'),
 array(['drum'], dtype='<U70'),
 array(['journey', 'long'], dtype='<U70'),
 array(['ady', 'cindy'], dtype='<U70'),
 array(['1999'], dtype='<U70'),
 array(['landscapes', 'wintry'], dtype='<U70'),
 array(['platinum', 'the', 'collection'], dtype='<U70'),
 array(['florida', 'lake', 'orange'], dtype='<U70'),
 array(['you', 'find', 'until'], dtype='<U70'),
 array(['breakthrough']

In [ ]:
!head -n 10 translated.txt

fb:m.0n1vy1h|###|brandon kramer|###|people|||person|||gender|###|dude guy male man men boy|###|fb:m.05zppz
fb:m.038b5bg|###|clean your room|###|music|||release|||track|###|des moines|###|fb:m.010btzs
fb:m.0cz9079|###|turgut doğan şahin|###|soccer|||football_player|||position_s|###|midfield midfielder|###|fb:m.02nzb8
fb:m.03d1n81|###|rollo john oliver meyer jack meyer|###|organization|||organization_founder|||organizations_founded|###|millfield|###|fb:m.04hk45
fb:m.03byqr1|###|mare nostrum|###|cvg|||computer_videogame|||cvg_genre|###|tactical shooter|###|fb:m.0_678
fb:m.0n5xzdf|###|david caffey|###|people|||person|||place_of_birth|###|shelby county / memphis city memphis , tennessee memphis|###|fb:m.0c_m3
fb:m.02vwbkk|###|luboš kalouda|###|people|||person|||nationality|###|czechoslovakia|###|fb:m.01mk6
fb:m.02hl5l7|###|spellbreaker|###|cvg|||game_version|||platform|###|msdos dos ms dos ms-dos microsoft disk operating system microsoft dos|###|fb:m.050xd
fb:m.0b70yxd|###|always goodbye|##

In [ ]:
def query(entity='brandon kramer', predicate='gender', subject_vectors = mid2vector, predicate_vectors=pred2vector):
    for index

In [ ]:
all_entities[:100]

['skyshaper',
 'collection',
 'gypsy 83',
 'show boat ( 1971 london revival cast )',
 'movie surfers',
 'die winterreise',
 'human rights quarterly',
 'remember me',
 'tommy ( london symphony orchestra & chamber choir )',
 'smack that',
 "lovely weather we 're having",
 'kansas city',
 'drum',
 'long , long journey',
 'cindy ady',
 '1999',
 'wintry landscapes',
 'the platinum collection',
 'orange lake , florida orange lake',
 'until i find you',
 'breakthrough',
 'daybreak',
 "heretic 's fork",
 'reese witherspoon',
 '1995',
 'doug sisk',
 'dyron nix',
 'kashif',
 'all good things',
 "edison 's conquest of mars",
 'salomon kalischer',
 'take it to the limit',
 'fantasy',
 'massachusetts',
 'morfiend',
 'franz ferdinand',
 'max tonetto',
 'still ( bonus disc : high wycombe hall live 20 february 1980 ) still',
 "the world 's greatest sinner",
 'oliver adler',
 'tan',
 'virtuoso',
 'star wars',
 'oliver waterman larkin oliver w. larkin',
 'walnut hill , philadelphia , pennsylvania walnut

In [ ]:

from BuboQA.scripts.util import www2fb, clean_uri
i = 1
fout = open('translated.txt', 'w')
with open('/home/textmining/HDD/mehdi_jafari/git/BERT-QA/BuboQA/data/SimpleQuestions_v2/freebase-subsets/freebase-FB2M.txt', 'r') as fin:
  for line in fin:
    if i % 500000 == 0:
      print("line: {}".format(i))
    i+=1
    # if i>=300:
    #   break 
    parts = line.strip().split('\t')
    if len(parts) != 3:
      print("ERROR: line - {}".format(line))
      continue 
    subject = www2fb(parts[0])
    predicate = www2fb(parts[1])
    object = www2fb(parts[2])
    try:
      tmp_subject, tmp_object = subject, object
      subject, object = ' '.join(mid2name[subject]), ' '.join(mid2name[object])
    except:
      continue
#     for s in subject:
#       for o in object:
    predicate = predicate.replace('.', '|||').replace('fb:', '')
    fout.write('{}|###|{}|###|{}|###|{}|###|{}\n'.format(tmp_subject, subject, predicate, object, tmp_object))
    
fout.close()

    
    

line: 500000
line: 1000000
line: 1500000
line: 2000000
line: 2500000
line: 3000000
line: 3500000
line: 4000000
line: 4500000
line: 5000000
line: 5500000
line: 6000000
line: 6500000
line: 7000000
line: 7500000
line: 8000000
line: 8500000
line: 9000000
line: 9500000
line: 10000000
line: 10500000


In [ ]:
!head -n 10 translated.txt

fb:m.0n1vy1h|###|brandon kramer|###|people|||person|||gender|###|dude guy male man men boy|###|fb:m.05zppz
fb:m.038b5bg|###|clean your room|###|music|||release|||track|###|des moines|###|fb:m.010btzs
fb:m.0cz9079|###|turgut doğan şahin|###|soccer|||football_player|||position_s|###|midfield midfielder|###|fb:m.02nzb8
fb:m.03d1n81|###|rollo john oliver meyer jack meyer|###|organization|||organization_founder|||organizations_founded|###|millfield|###|fb:m.04hk45
fb:m.03byqr1|###|mare nostrum|###|cvg|||computer_videogame|||cvg_genre|###|tactical shooter|###|fb:m.0_678
fb:m.0n5xzdf|###|david caffey|###|people|||person|||place_of_birth|###|shelby county / memphis city memphis , tennessee memphis|###|fb:m.0c_m3
fb:m.02vwbkk|###|luboš kalouda|###|people|||person|||nationality|###|czechoslovakia|###|fb:m.01mk6
fb:m.02hl5l7|###|spellbreaker|###|cvg|||game_version|||platform|###|msdos dos ms dos ms-dos microsoft disk operating system microsoft dos|###|fb:m.050xd
fb:m.0b70yxd|###|always goodbye|##

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
batch = 10000
class FreebaseKnowledgeBase:
    def __init__(self, path='translated.txt'):
        subjects = []
        predicates = []
        with open(path, 'r') as fin:
            for index, line in enumerate(fin):
                if index % 1000000 == 0:
                    print("line: {}".format(index))
                s_mid, subject, predicate, _, o_mid = line.strip().split('|###|')
                subjects.append(subject)
                predicates.append(predicate.replace('_', ' ').split('|||')[-1])
#                 if index>=100000:
#                     break 

        print('prob 1')  
        self.subject_vectorizer = TfidfVectorizer()
        self.predicate_vectorizer = TfidfVectorizer()
        self.subject_vectorizer.fit(subjects)
        self.predicate_vectorizer.fit(predicates)
        del subjects; del predicates
        
        temp_subjects = []
        temp_predicates = []
        temp_object_mids = []
        temp_subject_mids = []
        print('prob 2')  
        self.KB = []
        with open(path, 'r') as fin:
            for index, line in enumerate(fin):
                s_mid, subject, predicate, _, o_mid = line.strip().split('|###|')
                if index % 100000 == 0:
                    print("line: {}".format(index))
#                 if index>=100000:
#                     break 
                if len(temp_subjects)<=batch:
                    temp_subjects.append(subject)
                    temp_predicates.append(predicate.replace('_', ' ').split('|||')[-1])
                    temp_subject_mids.append(s_mid)
                    temp_object_mids.append(o_mid)
                    continue
                
                temp_subjects_vectors = self.subject_vectorizer.transform(temp_subjects)
                temp_predicates_vectors = self.predicate_vectorizer.transform(temp_subjects)
                temp_kb = [(s_mid, s_vector, p_vector, o) for s_mid, s_vector, p_vector, o in 
                            zip(temp_subject_mids, temp_subjects_vectors, temp_predicates_vectors, temp_object_mids)]
                
                self.KB+=temp_kb
                
                temp_subjects = []
                temp_predicates = []
                temp_object_mids = []
                temp_subject_mids = []
                

        
        


In [ ]:
a = FreebaseKnowledgeBase()

line: 0
line: 1000000
line: 2000000
line: 3000000
line: 4000000
line: 5000000
line: 6000000
line: 7000000
line: 8000000
line: 9000000
line: 10000000
prob 1
prob 2
line: 0
line: 100000
line: 200000
line: 300000
line: 400000
line: 500000
line: 600000
line: 700000
line: 800000
line: 900000
line: 1000000
line: 1100000
line: 1200000
line: 1300000
line: 1400000
line: 1500000
line: 1600000
line: 1700000
line: 1800000
line: 1900000
line: 2000000
line: 2100000
line: 2200000
line: 2300000
line: 2400000
line: 2500000
line: 2600000
line: 2700000
line: 2800000
line: 2900000
line: 3000000
line: 3100000
line: 3200000
line: 3300000
line: 3400000
line: 3500000
line: 3600000
line: 3700000
line: 3800000
line: 3900000
line: 4000000
line: 4100000
line: 4200000
line: 4300000
line: 4400000
line: 4500000
line: 4600000
line: 4700000
line: 4800000
line: 4900000
line: 5000000
line: 5100000
line: 5200000
line: 5300000
line: 5400000
line: 5500000
line: 5600000
line: 5700000
line: 5800000
line: 5900000
line: 600000

In [ ]:
a.KB[0]

NameError: name 'a' is not defined

In [ ]:
import sys

sys.getsizeof(a.KB)

93339448

In [ ]:
kb = a.KB
del a

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(0, 1000):
    print(cosine_similarity(kb[i][2], kb[0][2]).flatten())

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]


In [ ]:
def query(subject='brandon kramer', predicate='gender', KB=kb, candidates=50):
    entity_candidates = []
    

('fb:m.0n1vy1h',
 <1x650791 sparse matrix of type '<class 'numpy.float64'>'
 	with 2 stored elements in Compressed Sparse Row format>,
 <1x3062 sparse matrix of type '<class 'numpy.float64'>'
 	with 0 stored elements in Compressed Sparse Row format>,
 'fb:m.05zppz')